#Fine-tuning Bert for the HUHU Shared Task
##Approach
In this approach we trained, for subtask 1 and each of the categories of subtask 2A, one fine-tuned version of the bert_base_multilingual_cased model for binary classification.<br>
The code to implement the network was adapted from [this example](https://github.com/marcellusruben/medium-resources/blob/main/Text_Classification_BERT/bert_medium.ipynb).<br>
The Shared Task consisted of the following three subtasks (as explained in the official website).
##Subtask 1: HUrtful HUmour Detection
The first subtask consists in determining whether a prejudicial tweet is intended to cause humour. Participants will have to distinguish between tweets that using humour express prejudice and tweets that express prejudice without using humour.
##Subtask 2A: Prejudice Target Detection
Taking into account the following minority groups: Women and feminists, LGBTIQ community and Immigrants, racially discriminated people, and overweight people, participants are asked to identify the targeted groups on each tweet as a multilabel classification task.
##Subtask 2B: Degree of Prejudice Prediction
The third subtask consists of predicting on a continuous scale from 1 to 5 to evaluate how prejudicial the message is on average among minority groups.

##Imports

To run the following code, it is necessary to have the following files in the root of your directory structure:

*   huhu_split_train.csv
*   huhu_split_dev.csv

In our experiments, we decided to make our own internal 90%-10% split of the training set that was made available by the organizers. This training set can be downloaded from [zenodo](https://zenodo.org/record/7967255).

In [ ]:
import pandas as pd

train_set = pd.read_csv('huhu_split_train.csv')
dev_set = pd.read_csv('huhu_split_dev.csv')

In [ ]:
!pip install transformers

import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error

##Datasets

###Dataset (Task 1 and 2A)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
labels = {0:0,
          1:1
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, train_set, label):

        self.labels = [labels[iterLabel] for iterLabel in train_set[label]]
        self.texts = [tokenizer(text,
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in train_set['tweet']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

###Validation set

In [ ]:
np.random.seed(112)
train_set_humor, val_set_humor = np.split(train_set.sample(frac=1, random_state=42),
                                     [int(.9*len(train_set))])

print(len(train_set_humor),len(val_set_humor), len(dev_set))

##BertClassifier

In [ ]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

##Training loop

###Tasks 1 and 2A

In [ ]:
def train(model, train_data, val_data, label, learning_rate, epochs):

    train, val = Dataset(train_data, label), Dataset(val_data, label)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)

                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()

                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()

            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()

                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc

            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')

###Train

####Task 1 and 2A
The possible labels are: 'humor', 'prejudice_woman', 'prejudice_lgbtiq', 'prejudice_inmigrant_race' and 'prejudice_gordofobia'.

In [ ]:
EPOCHS = 5 #@param {type: "integer"}
model = BertClassifier()
LR = 1e-6 #@param {type: "number"}
label = 'humor' #@param {type: "string"}
train(model, train_set_humor, val_set_humor, label, LR, EPOCHS)

##Evaluation code

###Task 1 and 2A

In [ ]:
def evaluate(model, test_data, label):

    test = Dataset(test_data, label)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        prediction = []

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc

              prediction.append(output.argmax(dim=1).cpu().numpy())

    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    print("F1-Score Macro: " + str(round(f1_score(dev_set[label], prediction, average='macro')*100, 2)))
    scores = f1_score(dev_set[label], prediction, average=None, labels=[0,1])
    for i in range(len(labels)):
        print("F-Score for class (" + str(labels[i]) + "): " + str(round(scores[i]*100, 2)))
    print("F1-Score Positive: " + str(round(scores[1]*100, 2)))
    print("F1-Score Negative: " + str(round(scores[0]*100, 2)))

###Evaluate

####Task 1 and 2A
The possible labels are: 'humor', 'prejudice_woman', 'prejudice_lgbtiq', 'prejudice_inmigrant_race' and 'prejudice_gordofobia'.

In [ ]:
label = 'humor' #@param {type: "string"}
evaluate(model, dev_set, label)

###Save Model

In [ ]:
PATH = '' #@param {type: "string"}

model_name = '' #@param {type: "string"}

torch.save(model.state_dict(), PATH + model_name)

###Load Model

In [ ]:
PATH = '' #@param {type: "string"}
model_name = '' #@param {type: "string"}

model = BertClassifier()
model.load_state_dict(torch.load(PATH + model_name))

evaluate_on_load = True #@param {type: "boolean"}

if evaluate_on_load:
  label = 'humor' #@param {type: "string"}
  evaluate(model, dev_set, label)

##Inference

####Dataset Inference
This new dataset removes the label as it is not required.

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

class DatasetInference(torch.utils.data.Dataset):

    def __init__(self, data_set):
        self.texts = [tokenizer(text,
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in data_set['tweet']]

    def __len__(self):
        return len(self.texts)

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)

        return batch_texts

####Inference Code
The following code, given a dataframe with texts on the column 'tweet' and a model, will generate a list with the corresponding predictions for the texts presented.

In [ ]:
import numpy

def inference(model, data):
  test = DatasetInference(data)

  test_dataloader = torch.utils.data.DataLoader(test, batch_size=1)

  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda" if use_cuda else "cpu")

  if use_cuda:

      model = model.cuda()

  with torch.no_grad():

      prediction = []

      for test_input in test_dataloader:

            mask = test_input['attention_mask'].to(device)
            input_id = test_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)

            prediction.append(output.argmax(dim=1).cpu().numpy())

  return numpy.concatenate( prediction, axis=0 )

In [ ]:
prediction = inference(model,dev_set)

Run the next cell with the the previous dataset on the second line to get the texts with their respective prediction.

In [ ]:
results = pd.DataFrame()
results['tweet'] = dev_set['tweet']
results['inference'] = prediction

results